# 4.3 Exploratory analysis of ASF S1 imagery

Now that we have read in and organized the stack of Sentinel-1 RTC images, let's take a look at the data.

:::{admonition} ASF data access options
The steps shown in this notebook involve downloading and extracting large volumes of data. **It is not necessary to do this to follow the rest of the content in the tutorial**. We include the demonstration for the purposes of completeness and to help users who may be in this situation.

For more information on different options for downloading data locally, see the [Introduction](../s1_intro.md#different-ways-to-use-this-tutorial).
:::


::::{tab-set}
:::{tab-item} Outline

(content.section_A)=
**[A. Read and prepare data](#a-read-and-prepare-data)**  
- 1) Clip to spatial area of interest  

(content.section_B)=
**[B. Layover-shadow map](#b-layover-shadow-map)**  
- 1) Interactive visualization of layover-shadow maps

(content.section_C)=
**[C. Orbital direction](#c-orbital-direction)**  
- 1) Is a pass ascending or descending?
- 2) Assign orbital direction as a coordinate variable

(content.section_D)=
**[D. Duplicate time steps](#d-duplicate-time-steps)**  
- 1) Identify duplicate time steps
- 2) Visualize duplicates
- 3) Drop duplicates

(content.section_E)=
**[E. Examine coverage over time series](#e-examine-coverage-over-time-series)**

(content.section_F)=
**[F. Data Visualization](#f-data-visualization)**
- 1) Mean backscatter over time
- 2) Seasonal backscatter variability
- 3) Backscatter time series

:::
:::{tab-item} Learning goals  
#### Concepts
- Spatial joins of raster and vector data.  
- Visualize raster data.  
- Use raster metadata to aid interpretation of backscatter imagery.  
- Examine data quality using provided layover-shadow maps.  
- Identify and remove duplicate time step observations.  

#### Techniques
- Clip raster data cube using vector data with [`rioxarray.clip()`](https://corteva.github.io/rioxarray/html/examples/clip_geom.html).  
- Using `xr.groupby()` for [grouped statistics](https://docs.xarray.dev/en/stable/user-guide/groupby.html).  
- Reorganizing data with `xr.Dataset.reindex()`.  
- Visualizing multiple facets of the data using `FacetGrid`


:::
::::

:::{admonition} ASF Data Access
You can download the RTC-processed backscatter time series [here](https://zenodo.org/record/7236413#.Y1rNi37MJ-0). For more detail, see [tutorial data](../../background/4_tutorial_data.md#sentinel-1-rtc-datasets) and the [notebook](1_read_asf_data.ipynb) on reading ASF Sentinel-1 RTC data into memory.
:::

In [ ]:
# %xmode minimal
import geopandas as gpd
import hvplot.xarray
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import warnings
import xarray as xr

import s1_tools

warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
cwd = pathlib.Path.cwd()
tutorial2_dir = pathlib.Path(cwd).parent

{{break}}

## A. Read and prepare data

We'll go through the steps shown in [metadata wrangling](2_wrangle_metadata.ipynb), but this time,  combined into one function from `s1_tools`. 

:::{attention} 
If you are following along on your own computer, you **must** specify `'timeseries_type'` below: 
1. Set `timeseries_type` to `'full'` or `'subset'` depending on if you are using the full time series (103 files) or the subset time series (5 files).
:::

In [ ]:
timeseries_type = "full"

In [ ]:
vv_vrt_path = f"../data/raster_data/{timeseries_type}_timeseries/vrt_files/s1_stack_vv.vrt"
vh_vrt_path = f"../data/raster_data/{timeseries_type}_timeseries/vrt_files/s1_stack_vh.vrt"
ls_vrt_path = f"../data/raster_data/{timeseries_type}_timeseries/vrt_files/s1_stack_ls_map.vrt"

In [ ]:
asf_data_cube = s1_tools.metadata_processor(
    vv_path=vv_vrt_path,
    vh_path=vh_vrt_path,
    ls_path=ls_vrt_path,
    timeseries_type=timeseries_type,
)

In [ ]:
asf_data_cube

### 1) Clip to spatial area of interest

Until now, we've kept the full spatial extent of the dataset. This hasn't been a problem because all of our operations have been lazy. Now, we'd like to visualize the dataset in ways that require eager instead of lazy computation. We subset the data cube to a smaller area to focus on a location interest to make computation more less computationally-intensive.

Later notebooks use a different Sentinel-1 RTC dataset that is accessed for a smaller area of interest. Clip the current data cube to that spatial footprint:

In [ ]:
# Read vector data
pc_aoi = gpd.read_file("../data/vector_data/hma_rtc_aoi.geojson")

Visualize location

In [ ]:
pc_aoi.explore()

Check the CRS and ensure it matches that of the raster data cube:

In [ ]:
assert asf_data_cube.rio.crs == pc_aoi.crs, f"Expected: {asf_data_cube.rio.crs}, received: {pc_aoi.crs}"

Clip the raster data cube by the extent of the vector:

In [ ]:
clipped_cube = asf_data_cube.rio.clip(pc_aoi.geometry, pc_aoi.crs)
clipped_cube

Use [`xr.Dataset.persist()`](https://docs.xarray.dev/en/latest/generated/xarray.Dataset.persist.html); this method is an integration of Dask with Xarray. It will trigger the background computation of the operations we've so far executed lazily. Persist is similar to compute but it keeps the underlying data as dask-backed arrays instead of converting them to NumPy arrays.

In [ ]:
clipped_cube = clipped_cube.persist()

In [ ]:
clipped_cube.compute()

Great, we've gone from an object where each 3-d variable is ~ 90 GB to one where each 3-d variable is ~45 MB, this will be much easier to work with.

## B. Layover-shadow map


As discussed in previous notebooks, every Sentinel-1 scene comes with an associated layover shadow mask GeoTIFF file. This map describes the presence of layover, shadow and slope angle conditions that can impact backscatter values in a scene, which is especially important to consider in high-relief settings with more potential for geometric distortions

The following information is copied from the README file that accompanies each scene: 

```
Layover-shadow mask

The layover/shadow mask indicates which pixels in the RTC image have been affected by layover and shadow. This layer is tagged with _ls_map.tif

The pixel values are generated by adding the following values together to indicate which layover and shadow effects are impacting each pixel:
0.  Pixel not tested for layover or shadow
1.  Pixel tested for layover or shadow
2.  Pixel has a look angle less than the slope angle
4.  Pixel is in an area affected by layover
8.  Pixel has a look angle less than the opposite of the slope angle
16. Pixel is in an area affected by shadow

There are 17 possible different pixel values, indicating the layover, shadow, and slope conditions present added together for any given pixel._

The values in each cell can range from 0 to 31:
0.  Not tested for layover or shadow
1.  Not affected by either layover or shadow
3.  Look angle < slope angle
5.  Affected by layover
7.  Affected by layover; look angle < slope angle
9.  Look angle < opposite slope angle
11. Look angle < slope and opposite slope angle
13. Affected by layover; look angle < opposite slope angle
15. Affected by layover; look angle < slope and opposite slope angle
17. Affected by shadow
19. Affected by shadow; look angle < slope angle
21. Affected by layover and shadow
23. Affected by layover and shadow; look angle < slope angle
25. Affected by shadow; look angle < opposite slope angle
27. Affected by shadow; look angle < slope and opposite slope angle
29. Affected by shadow and layover; look angle < opposite slope angle
31. Affected by shadow and layover; look angle < slope and opposite slope angle

```

The ASF RTC image [product guide](https://hyp3-docs.asf.alaska.edu/guides/rtc_product_guide/) has detailed descriptions of how the data is processed and what is included in the processed dataset.

The `layover-shadow` variable provides categorical information so we'll use a qualitative colormap to visualize it.

In [ ]:
clipped_cube.isel(acq_date=1).ls.plot()

In [ ]:
cat_cmap = plt.get_cmap("tab20b", lut=32)
time_step1 = "2021-06-07"
time_step2 = "2021-06-10"

if timeseries_type == "subset":
    time_step1 = "2021-05-05T00:03:07"
    time_step2 = "2021-05-14T12:13:49"

fig, axs = plt.subplots(ncols=2, figsize=(12, 7), layout="constrained")

clipped_cube.sel(acq_date=time_step1).ls.plot(ax=axs[0], cmap=cat_cmap, cbar_kwargs=({"label": None}), vmin=0, vmax=31)

clipped_cube.sel(acq_date=time_step2).ls.plot(ax=axs[1], cmap=cat_cmap, cbar_kwargs=({"label": None}), vmin=0, vmax=31)

fig.suptitle(f"Layover shadow map of two time steps: {time_step1}, and {time_step2}", y=1.05)
for i in range(len(axs)):
    axs[i].set_xlabel(None)
    axs[i].set_ylabel(None)
    axs[i].tick_params(axis="x", labelrotation=45)

axs[0].set_title(time_step1)
axs[1].set_title(time_step2)
fig.supylabel("y coordinate of projection (m)")
fig.supxlabel("x coordinate of projection (m)");

It looks like there are areas affected by different types of distortion on different dates. For example, in the lower left quadrant, there is a region that is blue (5 - affected by layover) on 6/7/2021 but much of that area appear yellow (16 - affected by radar shadow) on 6/10/2021. This pattern is present throughout much of the scene with portions of area that are affected by layover in one acquisition in shadow in the next acquisition. This is not due to any real changes on the ground that occurred between the two acquisitions, rather it is the different viewing geometries of the orbital passes: one of the above scenes was collected during an ascending pass of the satellite and one during a descending pass. Since Sentinel-1 is always looking to the same side, ascending and descending passes will view the same area on the ground from opposing perspectives.

:::{attention}
If you're following along using the subset time series, your plot will look different than the plot above. That plot should display the layover-shadow map from 05/05/2021 on the left and 05/14/2021 on the right. In this plot, you'll see that the areas in shadow on 05/05/2021 are similar to the areas affected by layover on 05/14/2024.
:::

### 1) Interactive visualization of layover-shadow maps
We can use Xarray's integration with [hvplot](https://hvplot.holoviz.org/) (a library within the [holoviz](https://hvplot.holoviz.org/index.html) ecosystem) to look at the time series of layover-shadow maps interactively. Read more about interactive plots with Xarray and hvplot [here](https://tutorial.xarray.dev/intermediate/hvplot.html).

To do this, we need to demote `'ls'` from a coordinate variable to a data variable because of how `hvplot.xarray` expects the data to be structured. We can do this with [`xr.reset_coords()`](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.reset_coords.html#xarray.Dataset.reset_coords). First, recreate the above layover-shadow plot using hvplot:

In [ ]:
ls_var = clipped_cube.reset_coords("ls")

(
    ls_var.ls.sel(acq_date=time_step1)
    .squeeze()
    .hvplot(
        cmap="tab20b",
        width=400,
        height=350,
        clim=(0, 32),  # specify the limits of the colorbar to match original
        title=f"Acq date: {time_step1}",
    )
    + ls_var.ls.sel(acq_date=time_step2)
    .squeeze()
    .hvplot(
        cmap="tab20b",
        width=400,
        height=350,
        clim=(0, 32),
        title=f"Acq date: {time_step2}",
    )
)

(content:orbital_dir_section)=
## C. Orbital direction

Sentinel-1 is a right-looking sensor and it images areas on Earth’s surface in orbits when it is moving N-S (a descending orbit) and S-N (an ascending orbit). For a given scene, it images the same footprint on both passes but from different directions. The data coverage map below illustrates these directional passes with ascending passes moving from southeast to northwest and descending passes moving from northeast to southwest , it can be found online [here](https://asf.alaska.edu/daac/sentinel-1-acquisition-maps/).

```{image} ../imgs/slc_coverage_asf.png
:align center
```
ASF Sentinel-1 Cumulative coverage map.

In areas of high-relief topography such as the area we’re observing, there can be strong terrain distortion effects such as layover and shadow. These are some of the distortions that RTC processing corrects, but sometimes it is not possible to reliably extract backscatter for correction in the presence of strong distortions. The above image shows the layover-shadow map for an ascending and a descending image side-by-side, which is why different areas are affected by layover (5) and shadow (17) in each.

Thanks to all the setup work we did in the previous notebook, we can quickly confirm that all of the observations were taken at two times of day, corresponding to ascending and descending passes of the satellite, and that the time steps shown above were taken at different times of day.

:::{note}
The acquisition time of Sentinel-1 images is not in local time.
:::

In [ ]:
print(
    f"Hour of day of acquisition {time_step1}: ",
    clipped_cube.sel(acq_date=time_step1).acq_date.dt.hour.data,
)
print(
    f"Hour of day of acquisition {time_step2}: ",
    clipped_cube.sel(acq_date=time_step2).acq_date.dt.hour.data,
)
clipped_cube.acq_date.dt.hour

### 1) Is a pass ascending or descending?

In this example, it was relatively simple to determine one pass from another, but it is less straightforward to know if a pass is ascending or descending. The timing of these passes depends on the location on earth of the image. 

In the location covered by this dataset, ascending passes correspond to an acquisition time roughly 0:00 UTC and descending passes correspond to approximately 12:00 UTC. 

### 2) Assign orbital direction as a coordinate variable
This is another example of time-varying metadata, so it should be stored as a coordinate variable. Use [`xr.where()`](https://docs.xarray.dev/en/stable/generated/xarray.where.html) to assign the correct orbital direction value depending on an observation's acquisition time and then assign it as a coordinate variable to the clipped raster data cube.

In [ ]:
clipped_cube.coords["orbital_dir"] = (
    "acq_date",
    xr.where(clipped_cube.acq_date.dt.hour.data == 0, "asc", "desc"),
)

## D. Duplicate time steps

:::{note}
If you're working with the subset time series, skip this section.
:::

If we take a closer look at the ASF dataset, we can see that there are a few scenes from identical acquisitions (this is apparent in `acq_date` and more specifically in `product_id`). Let's examine these and see what's going on. 

:::{note}
This section **will not work** if you're using the subset timeseries.
:::

### 1) Identify duplicate time steps

First we'll extract the `data_take_ID` from the Sentinel-1 granule ID: 

In [ ]:
clipped_cube.data_take_ID.data

Let's look at the number of unique elements using [`np.unique()`](https://numpy.org/doc/stable/reference/generated/numpy.unique.html).

In [ ]:
data_take_ids_ls = clipped_cube.data_take_ID.data.tolist()
data_take_id_set = np.unique(clipped_cube.data_take_ID)
len(data_take_id_set)

Interesting - it looks like there are only 96 unique elements. Let's figure out which are duplicates:

In [ ]:
def duplicate(input_ls):
    return list(set([x for x in input_ls if input_ls.count(x) > 1]))


duplicate_ls = duplicate(data_take_ids_ls)
duplicate_ls

These are the data take IDs that are duplicated in the dataset. We now want to subset the xarray object to only include these data take IDs: 

In [ ]:
asf_duplicate_cond = clipped_cube.data_take_ID.isin(duplicate_ls)
asf_duplicate_cond

In [ ]:
duplicates_cube = clipped_cube.where(asf_duplicate_cond == True, drop=True)
duplicates_cube

### 2) Visualize duplicates

Great, now we have a 12-time step Xarray object that contains only the duplicate data takes. Let's see what it looks like. We can use `xr.FacetGrid` objects to plot all of the arrays at once.

Before we make a FacetGrid plot, we need to make a change to the dataset. FacetGrid takes a column and expands the levels of the provided dimension into individual sub-plots (a small multiples plot). We're looking at the duplicate time steps, meaning the elements of the `acq_date` dimension are non-unique. FacetGrid expects unique values along the specified coordinate array. If we were to directly call: 
```python
fg = duplicates_cube.vv.plot(col="acq_date", col_wrap=4)
``` 
We would receive the following error: 
```
ValueError: Coordinates used for faceting cannot contain repeated (nonunique) values.
```


Renaming the dimensions of `duplicates_cube` with [`xr.rename_dims()`](https://docs.xarray.dev/en/latest/generated/xarray.Dataset.rename_dims.html) demotes the `acq_date` coordinate array to non-dimensional coordinate and replaces it with `step` an array of integers. Because these are unique, we can make a FaceGrid plot with the `step` dimension.

In [ ]:
duplicates_cube.rename_dims({"acq_date": "step"})

In [ ]:
fg = duplicates_cube.rename_dims({"acq_date": "step"}).vv.plot(col="step", col_wrap=4)

Interesting, it looks like there's only really data for the 0, 2, 4, 7 and 9 elements of the list of duplicates. It could be that the processing of these files was interrupted and then restarted, producing extra empty arrays.

### 3) Drop duplicates

To drop these arrays, extract the product ID (the only variable that is unique among the duplicates) of each array we'd like to remove.

In [ ]:
drop_ls = [1, 3, 5, 6, 8, 10, 11]

We can use xarray's `.isel()` method, `.xr.DataArray.isin()`, `xr.Dataset.where()`, and list comprehension to efficiently subset the time steps we want to keep: 

In [ ]:
drop_product_id_ls = duplicates_cube.isel(acq_date=drop_ls).product_id.data
drop_product_id_ls

Using this list, we want to drop all of the elements of `clipped_cube` where product ID is one of the values in the list.

In [ ]:
duplicate_cond = ~clipped_cube.product_id.isin(drop_product_id_ls)
clipped_cube = clipped_cube.where(duplicate_cond == True, drop=True)
clipped_cube

## E. Examine coverage over time series

The previous section showed that there are some scenes in the time series with no data over the area of interest. Let's see if there are any data coverage characteristics we should examine more closely in the dataset. There are a number of ways to do this but approach we'll use here is to first calculate the percentage of pixels containing data relative to the entire footprint for each time step:

In [ ]:
max_pixels = clipped_cube.ls.notnull().any("acq_date").sum(["x", "y"])
valid_pixels = clipped_cube.ls.count(dim=["x", "y"])
clipped_cube["cov"] = (valid_pixels / max_pixels) * 100

In [ ]:
fig, ax = plt.subplots()
fig.suptitle("Distribution of percentage of valid pixels for \n each time step of backscatter time series")
clipped_cube["cov"].plot.hist(ax=ax)
ax.set_xlabel("% Coverage")
ax.set_title(None)
ax.set_ylabel("Count");

In addition to the empty duplicate time steps we removed above, there are many other time steps with minimal coverage. This is likely because the original time series contains scenes from multiple satellite footprints, and some may have minimal coverage over the area of interest that was used to clip the time series. To check this, we can again use Xarray's interactive visualization tools, this time to create an animation that loops through the time series. We'll get a bunch of warnings and errors if we try to make an animation of a time series with empty time steps, so we'll first remove any empty time steps (likely also caused by satellite footprints in the original time series that don't cover the smaller area of interest) and look just at scenes with some data:

:::{admonition} Masking with dask-backed arrays
If we tried to use `xr.Dataset.where()` to mask and drop time steps where coverage is zero right now, we would get the following error: 
```
KeyError: 'Indexing with a boolean dask array is not allowed. This will result in a dask array of unknown shape. Such arrays are unsupported by Xarray.Please compute the indexer first using .compute()'
```
Because we only called `.persist()` earlier, the arrays of the data variables are still Dask arrays instead of NumPy arrays. Calling `compute()` or `.load()` will resolve this issue and allow us to drop empty time steps: 


In [ ]:
clipped_cube.load()

In [ ]:
clipped_cube = clipped_cube.where(clipped_cube.cov > 0.0, drop=True)

Now that we've dropped the empty time steps, we can look at the layover shadow map at a few time steps: 

In [ ]:
clipped_cube.ls.isel(acq_date=slice(0, 10)).plot(col="acq_date", col_wrap=5, cmap="tab20b", vmin=0, vmax=32);

In the small multiples plot above, it looks like there are two main viewing geometries in the time series, and that one  of them covers the area of interest well while the other does not. Let's remove the time steps where coverage is less than 100. We won't need the `'cov'` variable anymore so we can drop it as well:

In [ ]:
clipped_cube = clipped_cube.where(clipped_cube.cov == 100, drop=True).drop_vars("cov")

In [ ]:
clipped_cube

In [ ]:
clipped_cube.ls.isel(acq_date=slice(0, 10)).plot(col="acq_date", col_wrap=5, cmap="tab20b", vmin=0, vmax=32);

If we remake the plots of the layover-shadow maps for the first ten timesteps, we'll see that they have the same viewing geometry.

## F. Data visualization

Now that we've visualized and taken a closer look at metadata such as the layover-shadow map and orbital direction, let's focus on backscatter variability. In the plotting calls in this notebook, we'll use a function in `s1_tools.py` that applies a logarithmic transformation to the data. This makes it easier to visualize variability, however, it is important to apply it as a last step before visualizing and to not perform statistical calculations on the log-transformed data.

:::{admonition} A note on visualizing SAR data
The measurements that we're provided in the RTC dataset are in intensity, or power, scale. Often, to visualize SAR backscatter, the data is converted from power to normalized radar cross section (the backscatter coefficient). This is in decibel (dB) units, meaning a log transform has been applied. This transformation makes it easier to visualize variability but it is important not to calculate summary statistics on log-transformed data as it will be distorted. You can read more about these concepts [here](https://hyp3-docs.asf.alaska.edu/guides/introduction_to_sar/#sar-scale).
:::

### 1) Mean backscatter over time

This section examines approaches of plotting VV and VH backscatter side by side that can have important consequences when visualizing and interpreting data. 

Currently, VV and VH are two data variables of the data cube that both exist along x,y and time dimensions. If we plot them both as individual subplots, we'll see two subplots, each with their own colormap. If you look closely, you can see that the colormaps are not on the same scale. We need to be careful when interpreting these images and comparing backscatter in the VV and VH images in this situation

In addition, we know from the layover-shadow maps that different areas are affected by shadow and masked from the dataset in both ascending and descending passes. For a careful examination of backscatter, we'll look at the mean over time of ascending scenes and mean over time of descending scenes.

In [ ]:
# Subset time series by ascending and descending passes
asc_pass_cond = clipped_cube.orbital_dir == "asc"
asc_pass_obs = clipped_cube.sel(acq_date=asc_pass_cond)

desc_pass_cond = clipped_cube.orbital_dir == "desc"
desc_pass_obs = clipped_cube.sel(acq_date=desc_pass_cond)

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(12, 12), layout="constrained")

s1_tools.power_to_db(asc_pass_obs["vv"].mean(dim="acq_date")).plot(
    ax=ax[0][0], cmap=plt.cm.Greys_r, cbar_kwargs=({"label": "dB"})
)
s1_tools.power_to_db(asc_pass_obs["vh"].mean(dim="acq_date")).plot(
    ax=ax[0][1], cmap=plt.cm.Greys_r, cbar_kwargs=({"label": "dB"})
)

s1_tools.power_to_db(desc_pass_obs["vv"].mean(dim="acq_date")).plot(
    ax=ax[1][0], cmap=plt.cm.Greys_r, cbar_kwargs=({"label": "dB"})
)
s1_tools.power_to_db(desc_pass_obs["vh"].mean(dim="acq_date")).plot(
    ax=ax[1][1], cmap=plt.cm.Greys_r, cbar_kwargs=({"label": "dB"})
)

for i in range(len(ax[0])):
    for j in range(len(ax)):
        ax[j][i].set_ylabel(None)
        ax[j][i].set_xlabel(None)
        ax[j][i].tick_params(axis="x", labelrotation=45)
fig.suptitle(
    "Mean backscatter over time broken up by polarization and orbital direction",
    fontsize=16,
)
ax[0][0].set_title("VV - Ascending passes")
ax[0][1].set_title("VH - Ascending passes")
ax[1][0].set_title("VV - Descending passes")
ax[1][1].set_title("VH - Descending passes")
fig.supylabel("y coordinate of projection (m)")
fig.supxlabel("x coordinate of projection (m)");

There are two ways to fix the problem of mismatched color map scales: 


#### Specify min and max values in plotting call


We could normalize the backscatter ranges for both variables by manually specifying a minimum and a maximum across both. 

In [ ]:
global_min = np.array(
    [
        s1_tools.power_to_db(asc_pass_obs["vv"].mean(dim="acq_date")).min(),
        s1_tools.power_to_db(asc_pass_obs["vh"].mean(dim="acq_date")).min(),
        s1_tools.power_to_db(desc_pass_obs["vv"].mean(dim="acq_date")).min(),
        s1_tools.power_to_db(desc_pass_obs["vh"].mean(dim="acq_date")).min(),
    ]
).min()

global_max = np.array(
    [
        s1_tools.power_to_db(asc_pass_obs["vv"].mean(dim="acq_date")).max(),
        s1_tools.power_to_db(asc_pass_obs["vh"].mean(dim="acq_date")).max(),
        s1_tools.power_to_db(desc_pass_obs["vv"].mean(dim="acq_date")).max(),
        s1_tools.power_to_db(desc_pass_obs["vh"].mean(dim="acq_date")).max(),
    ]
).max()

fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(12, 12), layout="constrained")

s1_tools.power_to_db(asc_pass_obs["vv"].mean(dim="acq_date")).plot(
    ax=ax[0][0],
    cmap=plt.cm.Greys_r,
    vmin=global_min,
    vmax=global_max,
    cbar_kwargs=({"label": "dB"}),
)
s1_tools.power_to_db(asc_pass_obs["vh"].mean(dim="acq_date")).plot(
    ax=ax[0][1],
    cmap=plt.cm.Greys_r,
    vmin=global_min,
    vmax=global_max,
    cbar_kwargs=({"label": "dB"}),
)
s1_tools.power_to_db(desc_pass_obs["vv"].mean(dim="acq_date")).plot(
    ax=ax[1][0],
    cmap=plt.cm.Greys_r,
    vmin=global_min,
    vmax=global_max,
    cbar_kwargs=({"label": "dB"}),
)

s1_tools.power_to_db(desc_pass_obs["vh"].mean(dim="acq_date")).plot(
    ax=ax[1][1],
    cmap=plt.cm.Greys_r,
    vmin=global_min,
    vmax=global_max,
    cbar_kwargs=({"label": "dB"}),
)

for i in range(len(ax[0])):
    for j in range(len(ax)):
        ax[j][i].set_ylabel(None)
        ax[j][i].set_xlabel(None)
        ax[j][i].tick_params(axis="x", labelrotation=45)
fig.suptitle(
    "Mean backscatter over time broken up by polarization and orbital direction",
    fontsize=16,
)
ax[0][0].set_title("VV - Ascending passes")
ax[0][1].set_title("VH - Ascending passes")
ax[1][0].set_title("VV - Descending passes")
ax[1][1].set_title("VH - Descending passes")
fig.supylabel("y coordinate of projection (m)")
fig.supxlabel("x coordinate of projection (m)");

#### Expand dimensions


We could convert the VV and VH data from being represented as data variables to as elements of a band dimension. This would let us use Xarray's FacetGrid plotting that creates small multiples along a given dimension.

In [ ]:
clipped_cube_da = clipped_cube.to_array(dim="band")

Since `orbital_dir` is a coordinate that exists along the time dimension, we do not want to expand the dimensions of the dataset. Instead, use `groupby()` to separate those observations:

In [ ]:
f = s1_tools.power_to_db(clipped_cube_da.groupby("orbital_dir").mean(dim="acq_date")).plot(
    col="band", row="orbital_dir", cmap=plt.cm.Greys_r, cbar_kwargs={"label": "dB"}
)
f.axs[1][0].tick_params(axis="x", labelrotation=45)
f.axs[1][1].tick_params(axis="x", labelrotation=45)
f.fig.supylabel("y coordinate of projection (m)")
f.fig.supxlabel("x coordinate of projection (m)")
f.fig.suptitle(
    "Mean backscatter over time for VV and VH polarizations and \n ascending and descending passes with FacetGrid",
    fontsize=16,
    y=1.05,
)
f.fig.set_figheight(12)
f.fig.set_figwidth(12);

The above plot shows the areas impacted by shadow in both ascending and descending returns. The time of day of the acquisition and the orientation of the imaged terrain with respect to the sensor are the only differences between ascending and descending backscatter images. Some studies have even exploited the different timing of ascending and descending passes in order to examine diurnal processes as melt-freeze cycles in an alpine snowpack {cite}`lund_2020_snowmelt`.

#### Creating a gap-filled backscatter image

In the above plots, we separate the observations by orbital direction in order to compare specific imaging conditions (eg. time of day) and to be able to observe the timing and location of conditions like shadow that impact backscatter. This is a helpful approach when we want a detailed understanding of scattering conditions in a given area. At other times, we may be more interested in building a complete backscatter image. Below, we'll show the same plots as above but combining ascending and descending pass scenes instead of keeping them separate: 

In [ ]:
f = s1_tools.power_to_db(clipped_cube_da.mean(dim="acq_date")).plot(
    col="band", cmap=plt.cm.Greys_r, cbar_kwargs=({"label": "dB"})
)

f.fig.suptitle("Mean backscatter over time for VV and VH polarizations with FacetGrid")
f.fig.set_figheight(7)
f.fig.set_figwidth(12);

The area that we're looking at is in the mountainous region on the border between the Sikkim region of India and China. There are four north-facing glacier visible in the image, each with a lake at the toe. Bodies of water like lakes tend to appear dark in C-band SAR images because water is smooth with respect to the wavelength of the signal, meaning that most of the emitted signal is scattered away from the sensor. where surfaces that are rough at the scale of C-band wavelength, more signal is returned to the sensor and the backscatter image is brighter. For much more detail on interpreting SAR imagery, see the resources linked in the Sentinel-1 section of the [tutorial data](../../background/4_tutorial_data.md) page.

### 2) Seasonal backscatter variability

:::{note}
If you're working with the subset time series, this plot will not display correctly because only one season is represented.
:::

Now let's look at how backscatter may vary seasonally for a single polarization (for more on time-related GroupBy operations see the [Xarray User Guide](https://docs.xarray.dev/en/stable/user-guide/time-series.html#resampling-and-grouped-operations)). This is an example of a 'split-apply-combine' operation, where a dataset is split into groups (in this case, time steps are split into seasons), an operation is applied (in this case, the mean is calculated) and then the groups are combined into a new object.

In [ ]:
clipped_cube_gb = clipped_cube.groupby("acq_date.season").mean()

The temporal dimension of the new object has an element for each season rather than an element for each time step.

In [ ]:
clipped_cube_gb

In [ ]:
# order seasons correctly
clipped_cube_gb = clipped_cube_gb.reindex({"season": ["DJF", "MAM", "JJA", "SON"]})
clipped_cube_gb

Visualize mean backscatter in each season:

In [ ]:
fg_vv = s1_tools.power_to_db(clipped_cube_gb.vv).plot(col="season", cmap=plt.cm.Greys_r, cbar_kwargs=({"label": "dB"}));

In [ ]:
fg_vh = s1_tools.power_to_db(clipped_cube_gb.vh).plot(col="season", cmap=plt.cm.Greys_r, cbar_kwargs=({"label": "dB"}))

The glacier surfaces appear much darker during the summer months than other seasons, especially in the lower reaches of the glaciers. Like the lake surfaces above, this suggests largely specular reflection where no signal returns in the incident direction. It could be that during the summer months, enough liquid water is present at the glacier surface to produce this scattering. 

### 3) Backscatter time series

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))

s1_tools.power_to_db(clipped_cube["vv"].sel(acq_date=asc_pass_cond).mean(dim=["x", "y"])).plot.scatter(
    ax=ax, label="VV - asc", c="b", marker="o"
)
s1_tools.power_to_db(clipped_cube["vv"].sel(acq_date=desc_pass_cond).mean(dim=["x", "y"])).plot.scatter(
    ax=ax, label="VV - desc", c="b", marker="x"
)
s1_tools.power_to_db(clipped_cube["vh"].sel(acq_date=asc_pass_cond).mean(dim=["x", "y"])).plot.scatter(
    ax=ax, label="VH - asc", c="r", marker="o"
)
s1_tools.power_to_db(clipped_cube["vh"].sel(acq_date=desc_pass_cond).mean(dim=["x", "y"])).plot.scatter(
    ax=ax, label="VH - desc", c="r", marker="x"
)
fig.legend(loc="center right")

fig.suptitle("Backscatter variability over time by polarization and orbital direction")
ax.set_title(None)
ax.set_ylabel("dB")
ax.set_xlabel("Time");

To make an interactive plot of backscatter variability, use [`hvplot`](https://tutorial.xarray.dev/intermediate/hvplot.html):

In [ ]:
vv_plot = s1_tools.power_to_db(clipped_cube["vv"].mean(dim=["x", "y"])).hvplot.scatter(x="acq_date")
vh_plot = s1_tools.power_to_db(clipped_cube["vh"].mean(dim=["x", "y"])).hvplot.scatter(x="acq_date")

vv_plot * vh_plot

## Conclusion

In this notebook, we demonstrated how to use the data cube that we assembled in the previous notebooks. We saw various ways that having metadata accessible and attached to the correct dimensions of the data cube made learning about the dataset much smoother and more efficient than it would otherwise be. 

In the next notebook, we'll work with a different Sentinel-1 RTC dataset. We'll write this dataset to disk in order to use it in the final notebook of the tutorial, a comparison of two datasets. 

In [ ]:
if timeseries_type == "full":
    clipped_cube.to_zarr(
        f"../data//raster_data/{timeseries_type}_timeseries/intermediate_cubes/s1_asf_clipped_cube.zarr",
        mode="w",
    )
else:
    pass